<a href="https://colab.research.google.com/github/ntdthanh1409/Baitap/blob/master/finalresult.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import bs4
import requests


#  Mỗi website bạn cào là một thư mục mới
def create_project_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        print("Creating project: " + directory)



#  Tạo Queue (hàng đợi) và crawled files (nếu chưa được tạo)
def create_data_files(project_name, base_url):
    queue = project_name + '/queue.txt'
    crawled = project_name + '/crawled.txt'
    if not os.path.isfile(queue):
        write_file(queue, base_url)
    if not os.path.isfile(crawled):
        write_file(crawled, '')


#  Tạo file mới
def write_file(path, data):
    f = open(path, 'a')
    f.write(data)
    f.close()


#  Thêm dữ liệu vào file đã tồn tại
def append_to_file(path, data):
    with open(path, 'a') as file:
        file.write(data + '\n')


#Xoá nội dung trong file
def delete_file_contents(path):
    with open(path, 'w'):
        pass

#  Đọc file và chuyển file vào set
def file_to_set(file_name):
    ket_qua = set()
    with open(file_name, 'rt') as f:
        for line in f:
            ket_qua.add(line.replace('/n', ''))
    return ket_qua


#  lặp url qua set để không craw trùng, mỗi url sẽ là một dòng mới trong file
def set_to_file(links, file):
    delete_file_contents(file)
    for link in sorted(links):
        append_to_file(file, link)

In [ ]:
def crawl(url, destination_set, final_set, max_results):
  if url not in history:
    history.append(url)
    res = requests.get(url)
    ressoup = bs4.BeautifulSoup(res.text, features="html.parser")
    for a in ressoup.find_all('a'):
        link = a.get('href')
        if link == None:
          continue
        else:
          if link.startswith('/'):
            link = url + link
          if link.startswith(seed):
            destination_set.add(link)
            final_set.add(link)
            if len(final_set) == max_results:
              return False # Dừng khi đã đạt kết quả tối đa
          else:
            continue
    return True # dừng sau khi duyệt hết

In [ ]:
def crawl_set(parent_set, child_set, final_set, max_results):
  for link in parent_set:
    flag = crawl(link, child_set, final_set, max_results)
    if not flag:
      return False # Dừng khi đã đạt kết quả tối đa
  return True # dừng sau khi duyệt hết

In [ ]:
DEPTH = int(input("Độ sâu mà bạn muốn crawl: "))
MAX_RESULTS = int(input("Số link mà bạn muốn lấy: "))
seed = input("url khởi đầu: ")
final_set = set([seed])
set_0 = set([seed])
history = []
url_list = []
url_list.append(set_0)
for x in range(DEPTH):
  new_set = set()
  url_list.append(new_set)
for i in range(DEPTH):
  flag = crawl_set(url_list[i], url_list[i+1], final_set, MAX_RESULTS)
  if not flag: # Đã đạt được số kết quả tối đa hay chưa
    break

In [ ]:
for s in url_list:
  print(len(s))
print(len(final_set))
for i in final_set:
  append_to_file('crawled.txt',i)
